# Parte 1: PDFs Downloader

Este bloque de código descarga los pdfs de la página de indecopi filtrando las resoluciones que tengan que ver con cláusulas abusivas y el año según ingrese el usuario.

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import requests
import time

# Inicializar el controlador del navegador (debes tener un navegador compatible instalado)
driver = webdriver.Chrome()  # Asegúrate de tener ChromeDriver descargado y configurado

# URL de la página de búsqueda
url = 'https://servicio.indecopi.gob.pe/buscadorResoluciones/proteccion-consumidor.seam'

# Acceder a la página web
driver.get(url)

# Encontrar el campo de búsqueda por su atributo HTML (puedes inspeccionar el HTML para obtener el atributo adecuado)
search_input = driver.find_element('id', 'FormListado1:txtTextoBusqueda')

# Introducir la búsqueda en el campo
search_input.send_keys('clausula abusiva')

# Solicitar al usuario que ingrese un año
year = input("Por favor, ingrese el año de búsqueda: ")

# Convertir la entrada a un entero (si es necesario)
try:
    year = int(year)
except ValueError:
    print("Eso no parece ser un número válido.")

# Encontrar el botón para seleccionar un año
search_year_input = driver.find_element('id', 'FormListado1:cmd_vc_distcomboboxValue')
driver.execute_script(f"arguments[0].value = '{year}';", search_year_input)

# Encontrar el botón de búsqueda por su atributo HTML
search_button = driver.find_element('id', 'FormListado1:b_RealizaBusqueda')

# Hacer clic en el botón
search_button.click()

# Esperar a que la página cargue (puedes ajustar el tiempo de espera según sea necesario)
time.sleep(15)  # Espera hasta 15 segundos

# Obtener el primer elemento con la clase "paginacion"
pagination_element = driver.find_element('class name', 'paginacion')

# Obtener el elemento hijo <ul>
ul_element = pagination_element.find_element('tag name', 'ul')

# Obtener la cantidad de elementos hijos <li> en <ul>
li_elements = ul_element.find_elements('tag name', 'li')

# Calcular la cantidad de veces que se debe repetir el proceso (restando 4)
repetitions_count = len(li_elements) - 4

for i in range(2, 2 + repetitions_count):
    # Encontrar el link de la página i
    pagination_i_link = driver.find_element('id', f'FormListado3:testpList:j_id114:{i}:cmlPage')
    pagination_i_link.click()

    # Esperar a que la página cargue (puedes ajustar el tiempo de espera según sea necesario)
    time.sleep(7)  # Espera hasta 7 segundos

    # Realizar la descarga 25 veces
    for j in range(25):
        # Obtener el ID y nombre de archivo para esta iteración
        id_number = str(j)
        file_number = str(j + 1)

        # Construir el ID del elemento contenedor del PDF
        pdf_container_id = f'FormListado3:testpList:{id_number}:j_id144'
        
        # Verificar si el elemento con el ID especificado existe
        pdf_container_elements = driver.find_elements('id', pdf_container_id)

        if pdf_container_elements:
            # El elemento existe, puedes obtenerlo y continuar con tu lógica
            pdf_container = pdf_container_elements[0]

            # Encontrar la etiqueta <a> dentro de pdf_container
            pdf_link = pdf_container.find_element('tag name', 'a')

            # Obtener la URL del PDF
            pdf_url = pdf_link.get_attribute('href')

            # Ruta donde deseas guardar el PDF
            save_route = f'./indecopi_pdfs/resolution{year}_{i-1}_{file_number}.pdf'

            # Descargar el PDF utilizando la biblioteca 'requests' y guardar en la ruta especificada
            response = requests.get(pdf_url)

            if response.status_code == 200:
                with open(save_route, 'wb') as pdf_file:
                    pdf_file.write(response.content)
                print(f'PDF {file_number} descargado en {save_route}')
            else:
                print(f'No se pudo descargar el PDF {file_number}. Código de estado: {response.status_code}')
        else:
            # El elemento no existe, puedes detener la iteración
            # print(f'No se encontró el elemento con ID {pdf_container_id}')
            break

# Cerrar el navegador
driver.quit()
